# Zillow6

## Save long tab

Saves Zillow tables (prices and sales) as long form, adding FIPS codes when not previously available.

## Setup
Load packages, functions and data done in separate so I dont clutter this code.

In [2]:
%run Zillow00_init.ipynb

Function save_plotly(figure, filename)

Function clean_TS(dataframe, value_name)
Chose a value_name of your preference: 'index', 'price', 'Zri'...

Function load_zillow(geography ,tenure=all)
geography=('state'/'county'), tenure=('rentals'/'prices'/'all')

Function add_FIPS(df)

Function add_logprices(df)
Log prices are created from last column.



In [3]:
# all Zillow series for prices
load_zillow('county', 'prices')

C_Zhvi_SFR ../input/zhvi/County_Zhvi_SingleFamilyResidence.csv
C_Zhvi_All ../input/zhvi/County_Zhvi_AllHomes.csv
C_Zhvi_Bot ../input/zhvi/County_Zhvi_BottomTier.csv
C_Zhvi_Top ../input/zhvi/County_Zhvi_TopTier.csv


## Price series 

### Clean tables, make long form, add log prices and census divisions
I will work with dates from 2000, when coverage of Zillow is better.

In [4]:
# original table
C_Zhvi_All.head(3)

,RegionID,SizeRank,RegionName,RegionType,StateName,State,Metro,StateCodeFIPS,MunicipalCodeFIPS,1996-01-31,...,2019-06-30,2019-07-31,2019-08-31,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31
0,3101,0,Los Angeles County,County,CA,CA,Los Angeles-Long Beach-Anaheim,6,37,177723.0,...,632055.0,632683.0,634150.0,636295.0,639331.0,641908.0,646084.0,650608.0,657464.0,664495.0
1,139,1,Cook County,County,IL,IL,Chicago-Naperville-Elgin,17,31,150249.0,...,247549.0,247860.0,247905.0,247709.0,247416.0,247110.0,246836.0,246704.0,246694.0,246806.0
2,1090,2,Harris County,County,TX,TX,Houston-The Woodlands-Sugar Land,48,201,100475.0,...,197084.0,197701.0,198241.0,198605.0,199206.0,199748.0,200350.0,200569.0,201110.0,202143.0


In [5]:
def long_clean(dx, initial_date='2000-01-01'):
    
    df = dx.copy() # make copy = do not change original
    df = clean_TS(df, 'price') # long table
    add_FIPS(df) # add 5-digit FIPS 
    df = df[['RegionName', 'State', 'FIPS', 'Date', 'price']] # select columns
    df = df[df.Date>='2000-01-01'] # drop dates before 2000
    
    # Add columns: log prices, % changes
    df['logprice'] = np.log(df.price)
    df['pct_change'] = df.groupby('FIPS')['price'].apply(lambda x: x.pct_change())
    return df

---------
## Mid tier
a.k.a "all" or typical house

In [6]:
df_long_allCountyPrices = long_clean(C_Zhvi_All)
df_long_allCountyPrices

,RegionName,State,FIPS,Date,price,logprice,pct_change
136464,Nantucket County,MA,25019,2000-01-31,643249.0,13.374287,NaN
136465,San Francisco County,CA,06075,2000-01-31,451296.0,13.019879,NaN
136466,San Mateo County,CA,06081,2000-01-31,439758.0,12.993980,NaN
136467,Pitkin County,CO,08097,2000-01-31,575639.0,13.263236,NaN
136468,Santa Clara County,CA,06085,2000-01-31,390709.0,12.875718,NaN
...,...,...,...,...,...,...,...
827308,Ontonagon County,MI,26131,2020-03-31,35014.0,10.463503,-0.002734
827309,Tillman County,OK,40141,2020-03-31,33816.0,10.428689,0.002045
827310,Leslie County,KY,21131,2020-03-31,33400.0,10.416311,0.005267
827311,Phillips County,AR,05107,2020-03-31,33247.0,10.411720,0.001295


In [7]:
# Save result locally (to plot and regress)
df_long_allCountyPrices.to_csv('../output/df_long_allCountyPrices.csv',index=False)

---------
## Top tier

In [8]:
# clean table
df_long_TOPCountyPrices = long_clean(C_Zhvi_Top)

# Save local 
df_long_TOPCountyPrices.to_csv('../output/df_long_TOPCountyPrices.csv',index=False)

---------
## Sales Counties
All arms-length transactions of single family, condominium and cooperative properties. Note that this number is not available by "tier", only for all transactions. Includes REO (real estate that belongs to RE company - not successfully foreclosured) and auctions.

Not available per tier - only one series.

Latency problem: sales are usually reported by local governemnts with a lag from contract signing to sales recorded. Zillow "nowcasts" sales using historical latency. In sum, recent points in the series should not be used.

In [9]:
df = pd.read_csv('../input2/Sale_Counts_County.csv')
df.head(3)

,RegionID,RegionName,StateName,SizeRank,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,...,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03
0,3101,Los Angeles County,California,1,4184.0,4900.0,5320.0,5478.0,6272.0,5915.0,...,7061.0,7426.0,7543.0,6425.0,7168.0,6345.0,6628.0,5355.0,4700.0,NaN
1,139,Cook County,Illinois,2,5088.0,5606.0,5567.0,6099.0,6666.0,6277.0,...,7862.0,8199.0,7981.0,6567.0,6733.0,5168.0,6473.0,5126.0,4475.0,4368.0
2,1090,Harris County,Texas,3,4358.0,4512.0,4744.0,4950.0,4947.0,4317.0,...,5619.0,6122.0,5690.0,4565.0,4807.0,4298.0,5186.0,3769.0,4180.0,NaN


#### Add reference to FIPS
Sales table do not State or Municipal FIPS codes. I will take the reference form All home sales table.

In [10]:
# RegionID (Zillow) dictionary from All home prices table
dfA = C_Zhvi_All[['RegionID', 'StateCodeFIPS', 'MunicipalCodeFIPS']] 
add_FIPS(dfA) # add 5-digit FIPS 
dic_regions = dict(dfA[['RegionID', 'FIPS']].set_index('RegionID'))

# insert column with FIPS on Sales table
loc = df.columns.get_loc('SizeRank')
df.insert(loc=loc, column='FIPS', value=df.RegionID.map(dic_regions['FIPS'])) 

#### Counties without match
After making the correspondence, I noticed counties in both tables that do not match. Unfortunatelly, I find 3 out of 5 treated counties in NY are not listed on the number of transactions table, as documented bellow, when none of these names are listed for NY.

In [11]:
df[df.RegionName.isin(['Kings County', 'Queen County', 'Bronx County'])].iloc[:,:5]

,RegionID,RegionName,StateName,FIPS,SizeRank
399,580,Kings County,California,06031,400


In [12]:
## counties not found on Sales counties in NY
(C_Zhvi_All[(~C_Zhvi_All.RegionID.isin(df.RegionID)) & (C_Zhvi_All.StateName=='NY')].
 sort_values("RegionName").
 iloc[:,:5])

,RegionID,SizeRank,RegionName,RegionType,StateName
25,401,25,Bronx County,County,NY
214,2255,214,Dutchess County,County,NY
6,581,6,Kings County,County,NY
971,2979,973,Montgomery County,County,NY
9,1347,9,Queens County,County,NY


#### Proceed cleaning and saving
I cant use this table for NY, but I will save it in case I use for the other counties.

In [13]:
# drop columns without FIPS correspondence 
# (really small, no transactions, that is why they do not have prices series)
df = df[df.FIPS.notna()]
df = clean_TS(df, 'counts') # long table
df = df[['RegionName', 'StateName', 'FIPS', 'Date', 'counts']] # select columns
df = df[df.Date>='2000-01'] # drop dates before 2000 -  note here there is no "DAY"

# add columns
df['logcounts'] = np.log(df.counts)
df['pct_change'] = df.groupby('FIPS')['counts'].apply(lambda x: x.pct_change())

# check results
df.sort_values(['RegionName', 'Date'])

/Users/pelly/opt/miniconda3/envs/geo_plotnine/lib/python3.8/site-packages/pandas/core/series.py:679: RuntimeWarning:

divide by zero encountered in log



,RegionName,StateName,FIPS,Date,counts,logcounts,pct_change
1198,Abbeville County,South Carolina,45001,2008-03,4.0,1.386294,NaN
3815,Abbeville County,South Carolina,45001,2008-04,1.0,0.000000,-0.75
6432,Abbeville County,South Carolina,45001,2008-05,0.0,-inf,-1.00
9049,Abbeville County,South Carolina,45001,2008-06,3.0,1.098612,inf
11666,Abbeville County,South Carolina,45001,2008-07,3.0,1.098612,0.00
...,...,...,...,...,...,...,...
368706,Zavala County,Texas,48507,2019-11,1.0,0.000000,-0.50
371323,Zavala County,Texas,48507,2019-12,4.0,1.386294,3.00
373940,Zavala County,Texas,48507,2020-01,NaN,NaN,0.00
376557,Zavala County,Texas,48507,2020-02,NaN,NaN,0.00


In [14]:
# save local
df.to_csv('../output/df_long_SaleCountsCounty.csv',index=False)